In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/supply-chain-dataset/supply_chain_data.csv


- <a href='#1'>1. Introduction </a>
    - <a href='#1.1'>1.1 Objective</a>
    - <a href='#1.2'>1.2 About the data </a>
- <a href='#2'>2. Source and Clean data</a>
    - <a href='#2.1'>2.1 Import Dependencies</a>
    - <a href='#2.2'>2.2 Load and Understand the data</a>
    - <a href='#2.3'>2.3 Data Cleaning</a>
        - <a href='#2.3.1'>2.3.1 Check for duplicates</a>
        - <a href='#2.3.2'>2.3.2 Standardize Column Names</a>
        - <a href='#2.3.2'>2.3.3 Rename Column Names</a>
- <a href='#3'>3. SQL Data Exploration</a>
    - <a href='#3.1'>3.1 Set up and connect to database</a>
    - <a href='#3.2'>3.2 SQL Queries</a>
        - <a href='#3.2.1'>3.2.1 Product Analysis</a>
        - <a href='#3.2.2'>3.2.2 Procurement Analysis</a>
        - <a href='#3.2.3'>3.2.3 Production Analysis</a>
        - <a href='#3.2.3'>3.2.4 Logistic Analysis</a>

# <a id='1'>1. Introduction</a>

## <a id='1.1'>1.1 Objective</a>
This project aims to leverage SQL to analyze a comprehensive dataset encompassing various aspects of a supply chain for a product line consisting of skincare, haircare, and cosmetics. By examining key metrics and identifying trends, we will gain valuable insights into optimizing inventory management, procurement processes, and overall supply chain efficiency.

## <a id='1.2'>1.2 About the data</a>

The dataset has the following columns:

* **Product type**: The type of product. Options: skincare, haircare, or cosmetics.
* **SKU**: Unique alphanumeric identifiers for each product.
* **Price**: The selling price of each product.
* **Availability**: The quantity of each product that is available for sale.
* **Number of products sold**: The quantity of products sold.
* **Revenue generated**: The revenue generated by each product.
* **Customer demographics**: Information about customer demographics. Options: Female, Male, Non-binary, or Unknown.
* **Stock levels**: Number of units physically present in the store. Availability can be more than stock levels if additional units are on order or in transit, making them ready for sale but not yet physically present in the inventory.
* **Lead times/ Business lead time**: The time taken to replenish stock, measured in days. Includes shipping time.
* **Order quantities**: The amount of items ordered.
* **Shipping times**: The time required for shipping to the business.
* **Shipping carriers**: The different carriers used by the business for collections. Options: Carrier A, Carrier B, or Carrier C.
* **Shipping costs**: The costs of shipping what the business ordered. These include packaging, handling, and delivery charges.
* **Supplier name**: The names of different suppliers.
* **Location**: The locations of the suppliers. Options: Mumbai, Kolkata, Delhi, Bangalore, or Chennai.
* **Lead time/ Supplier lead time**: The time taken for the supplier to replenish stock, measured in days.
* **Production volumes**: The amount of items produced by each supplier.
* **Manufacturing lead time**: The time required for manufacturing by each supplier.
* **Manufacturing costs**: The costs associated with manufacturing.
* **Inspection results**: The outcome of product inspections. Options: Pending, Fail, or Pass.
* **Defect rates**: The percentage of defects found in the products.
* **Transportation modes**: The method used by the supplier to deliver goods to the business. Options: Road, Air, Rail, or Sea.
* **Routes**: The route used by the supplier for shipment.Options: Route B, Route C or Route A
* **Costs / transportation cost**: The costs associated with transport. These can include fuel, vehicle maintenance, driver wages, tolls, and insurance.

# <a id='2'>2. Source and Clean the data</a>

## <a id='2.1'>2.1 Import Dependencies</a>

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3


## <a id='2.2'>2.2 Load and Understand the data</a>

In [3]:
df = pd.read_csv('/kaggle/input/supply-chain-dataset/supply_chain_data.csv')
df

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,haircare,SKU95,77.903927,65,672,7386.363944,Unknown,15,14,26,...,Mumbai,18,450,26,58.890686,Pending,1.210882,Air,Route A,778.864241
96,cosmetics,SKU96,24.423131,29,324,7698.424766,Non-binary,67,2,32,...,Mumbai,28,648,28,17.803756,Pending,3.872048,Road,Route A,188.742141
97,haircare,SKU97,3.526111,56,62,4370.916580,Male,46,19,4,...,Mumbai,10,535,13,65.765156,Fail,3.376238,Road,Route A,540.132423
98,skincare,SKU98,19.754605,43,913,8525.952560,Female,53,1,27,...,Chennai,28,581,9,5.604691,Pending,2.908122,Rail,Route A,882.198864


In [4]:
df.shape

(100, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

## <a id='2.3'>2.3 Data Cleaning</a>

### <a id='2.3.1'>2.3.1 Check for duplicates</a>

In [6]:
df.duplicated().any()

False

### <a id='2.3.2'>2.3.2 Standardize Column Names</a>

In [7]:
df.columns = df.columns.str.lower().str.replace(' ','_')
df.columns

Index(['product_type', 'sku', 'price', 'availability',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics',
       'stock_levels', 'lead_times', 'order_quantities', 'shipping_times',
       'shipping_carriers', 'shipping_costs', 'supplier_name', 'location',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs'],
      dtype='object')

### <a id='2.3.3'>2.3.3 Rename Column Names</a>

In [8]:
df = df.rename(columns={'lead_times': 'business_lead_time', 'lead_time':'supplier_lead_time', 'costs': 'transportation_costs'})
df.columns

Index(['product_type', 'sku', 'price', 'availability',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics',
       'stock_levels', 'business_lead_time', 'order_quantities',
       'shipping_times', 'shipping_carriers', 'shipping_costs',
       'supplier_name', 'location', 'supplier_lead_time', 'production_volumes',
       'manufacturing_lead_time', 'manufacturing_costs', 'inspection_results',
       'defect_rates', 'transportation_modes', 'routes',
       'transportation_costs'],
      dtype='object')

# <a id='3'>3. SQL Data Exploration</a>


## <a id='3.1'>3.1 Set up and connect to database</a>

In [9]:
engine = create_engine('sqlite:///supply_chain_db')
conn = engine.connect()

# Write dataFrame to SQL database
df.to_sql('chain', engine, if_exists='replace', index=False)

100

In [10]:
%load_ext sql

# Connect to database
%sql sqlite:///supply_chain_db

In [11]:
%%sql
-- List all the tables in the database
SELECT
    name
FROM sqlite_master
WHERE type='table';

 * sqlite:///supply_chain_db
Done.


name
chain


## <a id='3.2'>3.2 SQL Queries</a>

### <a id='3.2.1'>3.2.1 Product Analysis</a>

#### <font color=darkcyan>3.2.1.1 Calculate the total revenue generated for each product type.</font>

In [12]:
%%sql
SELECT
  product_type,
  ROUND(SUM(revenue_generated),2) AS revenue_generated
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


product_type,revenue_generated
skincare,241628.16
haircare,174455.39
cosmetics,161521.27


#### <font color=darkcyan>3.2.1.2 Determine the average price of products in each product category.</font>

In [13]:
%%sql
SELECT
  product_type,
  ROUND(AVG(price),2) AS avg_price
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


product_type,avg_price
cosmetics,57.36
skincare,47.26
haircare,46.01


#### <font color=darkcyan>3.2.1.3 Identify the top 5 product with the highest number of units sold.</font>

In [14]:
%%sql
SELECT
  product_type,
  sku,
  SUM(number_of_products_sold) AS sales_volume
FROM chain
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 5;

 * sqlite:///supply_chain_db
Done.


product_type,sku,sales_volume
skincare,SKU10,996
cosmetics,SKU94,987
skincare,SKU9,980
skincare,SKU36,963
skincare,SKU37,963


#### <font color=darkcyan>3.2.1.4 Calculate the average amount of days it takes for a supplier to deliver products.</font>

In [15]:
%%sql
SELECT
  supplier_name,
  CAST(AVG(supplier_lead_time) AS INT) AS avg_lead_time
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


supplier_name,avg_lead_time
Supplier 3,20
Supplier 5,18
Supplier 2,18
Supplier 4,15
Supplier 1,14


#### <font color=darkcyan>3.2.1.5 Determine the top 3 products by sales volume for each product type.</font>

In [16]:
%%sql
WITH ProductSales AS (
  SELECT
    product_type,
    sku,
    SUM(number_of_products_sold) AS sales_volume
  FROM chain
  GROUP BY 1, 2
)

SELECT *
FROM
(
  SELECT *,
  ROW_NUMBER()OVER(
    PARTITION BY product_type
    ORDER BY sales_volume DESC) AS product_rnk
FROM ProductSales) AS sb1
WHERE product_rnk <= 3;


 * sqlite:///supply_chain_db
Done.


product_type,sku,sales_volume,product_rnk
cosmetics,SKU94,987,1
cosmetics,SKU44,919,2
cosmetics,SKU91,916,3
haircare,SKU78,946,1
haircare,SKU74,904,2
haircare,SKU22,884,3
skincare,SKU10,996,1
skincare,SKU9,980,2
skincare,SKU36,963,3


### <a id='3.2.2'>3.2.2 Procurement Analysis</a>

#### <font color=darkcyan>3.2.2.1 Calculate average lead times for different suppliers.</font>

In [17]:
%%sql
SELECT 
  supplier_name,
  ROUND(AVG(supplier_lead_time),2) AS avg_supplier_lead_time
FROM chain
GROUP BY 1;

 * sqlite:///supply_chain_db
Done.


supplier_name,avg_supplier_lead_time
Supplier 1,14.78
Supplier 2,18.55
Supplier 3,20.13
Supplier 4,15.22
Supplier 5,18.06


#### <font color=darkcyan>3.2.2.2 Identify the top 3 suppliers with the highest defect rates.</font>

In [18]:
%%sql
SELECT
  supplier_name,
  ROUND(AVG(defect_rates),2) AS avg_defect_rates
FROM chain
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 3;

 * sqlite:///supply_chain_db
Done.


supplier_name,avg_defect_rates
Supplier 5,2.67
Supplier 3,2.47
Supplier 2,2.36


#### <font color=darkcyan>3.2.2.3 Find the most used supplier for each product type.</font>

In [19]:
%%sql
WITH SupplierCounts AS (
SELECT
  product_type, 
  supplier_name, 
  COUNT(*) AS supplier_count,
  DENSE_RANK()OVER(
    PARTITION BY product_type
    ORDER BY COUNT(*) DESC
  ) AS supplier_rnk
FROM chain
GROUP BY 1 , 2)

SELECT *
FROM SupplierCounts
WHERE supplier_rnk = 1;

 * sqlite:///supply_chain_db
Done.


product_type,supplier_name,supplier_count,supplier_rnk
cosmetics,Supplier 5,7,1
cosmetics,Supplier 2,7,1
cosmetics,Supplier 1,7,1
haircare,Supplier 4,10,1
haircare,Supplier 2,10,1
skincare,Supplier 1,15,1


### <a id='3.2.3'>3.2.3 Production Analysis</a>

#### <font color=darkcyan>3.2.3.1 Calculate the total production volume for each supplier.</font>

In [20]:
%%sql
SELECT
  supplier_name,
  SUM(production_volumes) AS production_volumes
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


supplier_name,production_volumes
Supplier 2,14105
Supplier 1,13545
Supplier 4,11756
Supplier 5,9381
Supplier 3,7997


#### <font color=darkcyan>3.2.3.2 Determine the average manufacturing lead time in days across all products.</font>

In [21]:
%%sql
SELECT
  AVG(manufacturing_lead_time) AS avg_manufacturing_lead_time
FROM chain;

 * sqlite:///supply_chain_db
Done.


avg_manufacturing_lead_time
14.77


#### <font color=darkcyan>3.2.3.3 Identify the supplier, product and product type with the lowest defect rates.</font>

In [22]:
%%sql
SELECT
  supplier_name,
  sku,
  product_type,
  defect_rates
FROM chain
WHERE defect_rates = (SELECT MIN(defect_rates) FROM chain);

 * sqlite:///supply_chain_db
Done.


supplier_name,sku,product_type,defect_rates
Supplier 5,SKU21,cosmetics,0.0186075676310149


#### <font color=darkcyan>3.2.3.4 Calculate the average manufacturing cost per unit for each product type.</font>

In [23]:
%%sql
SELECT
  product_type,
  ROUND(SUM(manufacturing_costs) / SUM(production_volumes),2) AS avg_unit_cost
FROM chain
GROUP BY 1;

 * sqlite:///supply_chain_db
Done.


product_type,avg_unit_cost
cosmetics,0.09
haircare,0.08
skincare,0.08


#### <font color=darkcyan>3.2.3.5 Analyze the relationship between production volume and defect rates for each supplier.</font>

In [24]:
%%sql
SELECT
  supplier_name,
  SUM(production_volumes) AS production_volumes,
  ROUND(AVG(defect_rates), 2) AS avg_defect_rates
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


supplier_name,production_volumes,avg_defect_rates
Supplier 2,14105,2.36
Supplier 1,13545,1.8
Supplier 4,11756,2.34
Supplier 5,9381,2.67
Supplier 3,7997,2.47


### <a id='3.2.4'>3.2.4 Logistic Analysis</a>

#### <font color=darkcyan>3.2.4.1 Calculate the total shipping cost for each shipping carrier.</font>

In [25]:
%%sql
SELECT
  shipping_carriers,
  ROUND(SUM(shipping_costs),2) AS shipping_costs
FROM chain
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///supply_chain_db
Done.


shipping_carriers,shipping_costs
Carrier B,236.9
Carrier C,162.38
Carrier A,155.54


#### <font color=darkcyan>3.2.4.2 Determine the average shipping time for products delivered by air.</font>

In [26]:
%%sql
SELECT
  ROUND(AVG(shipping_times),2) AS avg_shipping_time
FROM chain
WHERE transportation_modes = 'Air';

 * sqlite:///supply_chain_db
Done.


avg_shipping_time
5.12


#### <font color=darkcyan>3.2.4.3 Determine the most frequently used transportation mode for products sourced from Mumbai.</font>

In [27]:
%%sql
SELECT
  transportation_modes,
  COUNT(*) AS mode_count
FROM chain
WHERE location = 'Mumbai'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;

 * sqlite:///supply_chain_db
Done.


transportation_modes,mode_count
Air,7


#### <font color=darkcyan>3.2.4.4 Identify the top 3 suppliers with the highest average shipping costs.</font>

In [28]:
%%sql
SELECT
  supplier_name,
  ROUND(AVG(shipping_costs),2) AS avg_shipping_costs
FROM chain
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3;

 * sqlite:///supply_chain_db
Done.


supplier_name,avg_shipping_costs
Supplier 5,5.79
Supplier 4,5.76
Supplier 2,5.74


#### <font color=darkcyan>3.2.4.5 Find the shipping routes with average shippingcosts exceeding the average shipping cost for all the routes.</font>

In [29]:
%%sql
SELECT
  routes,
  ROUND(AVG(shipping_costs),2) AS avg_shipping_cost
FROM chain
GROUP BY 1
HAVING ROUND(AVG(shipping_costs),2) > (SELECT ROUND(AVG(shipping_costs),2) FROM chain)

 * sqlite:///supply_chain_db
Done.


routes,avg_shipping_cost
Route C,5.9


#### <font color=darkcyan>3.2.4.6 Find the top carriers per location based on shipment volume.</font>

In [30]:
%%sql
WITH LocationCarriers AS(
SELECT
  location,
  shipping_carriers,
  COUNT(shipping_carriers) AS shipment_volume,
  RANK()OVER(
    PARTITION BY location
    ORDER BY COUNT(shipping_carriers) DESC
  )AS rnk
FROM chain
GROUP BY 1, 2
)

SELECT
  location,
  shipping_carriers,
  shipment_volume
FROM LocationCarriers
WHERE rnk = 1;

 * sqlite:///supply_chain_db
Done.


location,shipping_carriers,shipment_volume
Bangalore,Carrier B,10
Chennai,Carrier B,8
Delhi,Carrier B,6
Kolkata,Carrier C,9
Kolkata,Carrier B,9
Mumbai,Carrier B,10


#### <font color=darkcyan>3.2.4.7 Classify shipping carriers into performance categories (e.g., good, average, poor) by comparing their average shipping time to the overall average shipping time and calculate the total shipping cost for each carrier.</font>

In [31]:
%%sql
WITH AvgShippingTime AS (
  SELECT
    ROUND(AVG(shipping_times),2) AS avg_shipping_time
  FROM chain
),

CarrierPerformanceAndCosts AS (
  SELECT
    c.shipping_carriers,
    CASE
      WHEN AVG(shipping_times) < a.avg_shipping_time THEN 'Poor'
      WHEN AVG(shipping_times) > a.avg_shipping_time THEN 'Good'
      ELSE 'Average'
    END AS carrier_performance,
    ROUND(SUM(shipping_costs)) AS shipping_costs
  FROM chain AS c
  CROSS JOIN AvgShippingTime AS a
  GROUP BY 1
)

SELECT *
FROM CarrierPerformanceAndCosts;

 * sqlite:///supply_chain_db
Done.


shipping_carriers,carrier_performance,shipping_costs
Carrier A,Good,156.0
Carrier B,Poor,237.0
Carrier C,Good,162.0
